# <font color = lightblue> 엑셀 파일 설정

In [1]:
from openpyxl import *

In [2]:
search_key = "저출산 뉴스"    # 검색할 단어
excelfile = load_workbook('test.xlsx')    # 엑셀 파일 로드

In [3]:
excelfile = load_workbook('test.xlsx') # test.xlsx 파일을 load 

excelfile.create_sheet(index=3, title=search_key) # sheet를 검색할 단어 이름으로 생성 
sheet = excelfile[search_key] # 생성한 시트를 변수에 할당 
ti = sheet.cell(row=1, column=1) # 시트의 1행, 1열에 "영상 제목"이라는 문자열 삽입 
ti.value = "영상 제목" 

comment_list = sheet.cell(row=1, column=2) # 시트의 1행, 2열에 "댓글"이라는 문자열 삽입 
comment_list.value = "댓글" 
excelfile.save('./test.xlsx') # 파일 저장

# <font color = lightblue>스크래핑

#### <font color = pink> 검색어로 유튜브 검색

In [4]:
from selenium import webdriver
import time

#driver = webdriver.Chrome('C:/Users/ledu2/Python/Crawling/chromedriver.exe')
driver = webdriver.Chrome()    # 현재 폴더에 chromedriver.exe가 있어서 경로 설정X
driver.get('https://www.youtube.com/results?search_query='+search_key)
time.sleep(3)

#### <font color = pink> 아래로 스크롤해서 모든 동영상 로드

In [5]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 

#html = driver.find_element_by_tag_name('html')    # 이거 못씀. 문법 바뀜
html = driver.find_element(By.TAG_NAME, "html")    # 이거로 바뀜
prev_contents_len = 0 

while True: 
    time.sleep(3) 
    contents = driver.find_elements(By.XPATH, '//ytd-page-manager[@id="page-manager"]/ytd-search/div[@id="container"]/ytd-two-column-search-results-renderer/div[@id="primary"]/ytd-section-list-renderer/div[@id="contents"]/ytd-item-section-renderer/div[@id="contents"]/ytd-video-renderer') 
    current_contents_len = len(contents)
    print(current_contents_len)
    if current_contents_len >= 100:    # 영상이 100개 이상 로드되면 중지
        break 
    html.send_keys(Keys.END)

20
38
58
78
98
118


#### <font color = pink> 댓글 수집

In [59]:
from selenium.webdriver.common.by import By 
count = 0

contents = driver.find_elements(By.XPATH, '//ytd-page-manager[@id="page-manager"]/ytd-search/div[@id="container"]/ytd-two-column-search-results-renderer/div[@id="primary"]/ytd-section-list-renderer/div[@id="contents"]/ytd-item-section-renderer/div[@id="contents"]/ytd-video-renderer/div[@id="dismissible"]/ytd-thumbnail') 
for c in contents:
    html = driver.find_element(By.TAG_NAME, "html")    # 이거로 바뀜
    print(count)
    if count == 50:
        break
    count+=1
    c.click()
    time.sleep(3)
    title = driver.find_element(By.XPATH, '//*[@id="title"]/h1/yt-formatted-string').text
    print(title)
    html.send_keys(Keys.END)
    time.sleep(5) 
    new_contents = driver.find_element(By.XPATH, '//*[@id="comments"]')
    time.sleep(5) 
    header = new_contents.find_elements(By.XPATH, '//ytd-item-section-renderer[@id="sections"]/div[@id="header"]/ytd-comments-header-renderer/div[@id="title"]/h2[@id="count"]/yt-formatted-string/span[@dir="auto"]')
    print(len(header))
    #header = driver.find_elements(By.XPATH, '//*[@id="count"]/yt-formatted-string/span[2]')
    time.sleep(5)
    
    if len(header) == 0: 
        driver.back() 
        continue 
        
    comment_num = int(header[1].text.replace(',', ''))
    prev_comment_len = 0 
    
    while True:
        time.sleep(3) 
        comments = new_contents.find_elements(By.XPATH, '//ytd-item-section-renderer[@id="sections"]/div[@id="contents"]/ytd-comment-thread-renderer') 
        
        if prev_comment_len == len(comments): 
            break 
            
        prev_comment_len = len(comments) 
        html.send_keys(Keys.END) 
    comments = new_contents.find_elements(By.XPATH, '//ytd-item-section-renderer[@id="sections"]/div[@id="contents"]/ytd-comment-thread-renderer') 
    
    for comment in comments: 
        print(comment.find_element(By.ID, 'content-text').text)
        sheet.append([title, comment.find_element(By.ID, 'content-text').text]) 
        excelfile.save('./test.xlsx') 
        time.sleep(5) 
        driver.back() 
    #driver.quit()

0
[MBC 탐사기획 스트레이트 213회] 인공지능의 습격(23.06.04)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008E6E73+48323]
	(No symbol) [0x00879661]
	(No symbol) [0x00785308]
	(No symbol) [0x0076D243]
	(No symbol) [0x007CE51B]
	(No symbol) [0x007DC613]
	(No symbol) [0x007CA6D6]
	(No symbol) [0x007A847C]
	(No symbol) [0x007A957D]
	GetHandleVerifier [0x00B4FD5D+2575277]
	GetHandleVerifier [0x00B8F86E+2836158]
	GetHandleVerifier [0x00B896DC+2811180]
	GetHandleVerifier [0x009741B0+626688]
	(No symbol) [0x0088314C]
	(No symbol) [0x0087F4B8]
	(No symbol) [0x0087F59B]
	(No symbol) [0x008721B7]
	BaseThreadInitThunk [0x76A97D59+25]
	RtlInitializeExceptionChain [0x77A5B74B+107]
	RtlClearBits [0x77A5B6CF+191]


In [6]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import warnings 
warnings.filterwarnings('ignore')

count = 0

contents = driver.find_elements(By.XPATH, '//ytd-page-manager[@id="page-manager"]/ytd-search/div[@id="container"]/ytd-two-column-search-results-renderer/div[@id="primary"]/ytd-section-list-renderer/div[@id="contents"]/ytd-item-section-renderer/div[@id="contents"]/ytd-video-renderer/div[@id="dismissible"]/ytd-thumbnail') 
for c in contents:
    print(count)
    if count == 50:
        break
    count+=1
    c.click()
    time.sleep(3)
    title = driver.find_element(By.XPATH, '//*[@id="title"]/h1/yt-formatted-string').text
    print(title)
    html = driver.find_element(By.TAG_NAME, "html")    # 이거로 바뀜
    html.send_keys(Keys.END)
    time.sleep(3)
    
    before_location = driver.execute_script("return window.pageYOffset")
    while True:
        #현재 위치 + 500으로 스크롤 이동
        driver.execute_script("window.scrollTo(0,{})".format(before_location + 500))

        #전체 스크롤이 늘어날 때까지 대기
        time.sleep(2)

        #이동 후 스크롤 위치
        after_location = driver.execute_script("return window.pageYOffset")

        #이동후 위치와 이동 후 위치가 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
        if before_location == after_location:
            break

        #같지 않으면 다음 조건 실행
        else:
            #이동여부 판단 기준이 되는 이전 위치 값 수정
            before_location = driver.execute_script("return window.pageYOffset")

    try:
        driver.find_element(By.CSS_SELECTOR, "#dismiss-button > a").click()
    except:
        pass
    
    # 대댓글
    buttons = driver.find_elements(By.CSS_SELECTOR, "#more-replies > a")

    time.sleep(1.5)

    for button in buttons:
        button.send_keys(Keys.ENTER)
        time.sleep(1.5)
        button.click()
    #여기까지
    
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    id_list = soup.select("div#header-author > h3 > #author-text > span")
    comment_list = soup.select("yt-formatted-string#content-text")

    comment_final = []

    for i in range(len(comment_list)):
        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        print(temp_comment)
        comment_final.append(temp_comment) # 댓글 내용
        
    for idx, comment in enumerate(comment_final): 
        sheet.append([title, comment]) 
        excelfile.save('./test.xlsx') 
        print(idx)
    driver.back() 

0
미래로 넘기는 시한폭탄(연중기획 인구 2편) [풀영상] | 창 426회 (KBS 23.07.11)
제일 무서운 병은 말기가 될때까지 그부위에서 고통이 느껴지지 않는병이다고통이 찾아왔을때는 이미 치료할수없을때니까
좋은 다큐 감사합니다.
현재로 보면 나라가 망조로 가고 있음에 부인할 수  있는 사람이 얼마나 될까 정의롭지 못하고 불평등이 고착화되고 있는데 각자도생이다. 정말 안타깝지만 그게 현실이되고 있음에 불안하다.
사기, 횡령, 배임에 관대한 사법부가 초래한 사회전반의 저신뢰 풍토를 해소하지 않는 한어떤 사회 문제도 해결 불가능해요
한국인들의 착각. 일본을 타산지석으로 삼는다? 현실은 일본보다 엄청나게 심각하다.
kbs 창 제작자분들이 다른 시사프로보다 제일 똑똑하신것 같아요~! 통틀어 가장 좋아하는 프로그램~!
투표에서 알 수 있듯이 인간의 패시브는 이기적인 본능을 가지고 있음. 이제 노인이 젊은층보다 압도적으로 많아졌을때 한국의 문제는 최고점을 향할꺼라고 봄
일본 다시 살아나고 있습니다
왜 눈물이 날까...
국민연금은 빠른 시일 내에 사회적 타협이 이루어져야 합니다!!!
Kbs 다큐 진짜 열심히 한다. 응원합니다
시사기획창이 최고프로그램..
20년후가 문제입니다.원화자산을 외화자산으로 바꿔놓아야 할겁니다
열심히 산 사람들 등쳐먹는 사기꾼들에 대해 처벌을 강화하고 노동하는 사람들에 대해 처우를 개선해주고. 안락사에 대한 길도 열어줘라. 살고자 노동하는 사람은 살도록, 게으르게 산 사람은 선택하도록, 사기친 사람은 처벌빋도록
일단 대한민국 정치인들의 자식들은 2중 국적이나 다른국가 영주권 못받게 만들어야함. 분열로 돈벌어서 지 새끼들은 다른나라가서 살지못하게.
그 어떤 복지보다 영화속의 나오는 프렌 시스템이 압권이다 빨리 현실에서 적용되었음 한다
사람은 누구나 늙는다는 것을 항상 인지해야 함... 일단 사전연명의료의향서 같은 좋은제도를 많이 퍼뜨려서 쓸데없는 연명치료에 필요한 재원을 줄이는게 급선무라고 생각한다.
지금의 지방소멸 속도를보면 얼마나 심각한지를

8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
28

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
2
[오늘 이슈] “결혼 진짜 안 한다더니…” 25-49세 남성 절반이 ‘미혼’ / KBS 2023.07.11.
크 우리나라  남자들 응원합니다
퇴근하고 와서 넷플릭스 영드 미드 애니만 봐도 행복함 ㅋㅋ 결혼 그냥 포기했음
설거지론 퐁퐁남 퍼진 20대는 지금 혼인율이나 출산율보다도 절반은 더 줄어들 예정30.40대는 이미 모른 채 결혼한 사람들 많은 거고 앞으로 10년 뒤 20대들이 혼인시기 되었을 때 통계가 너무 기대된다 지금이 최악이라고 생각하지 말기를 지금 20대들한테서 얼마나 충격받으려고? ㅋㅋ
결혼한 두명의 친구가, 한명은 집의 명의를 와이프 명의로 바꾸어주었고, 다른 한명은 대출로 사서 집 명의는 와이프 대출은 자기 이름으로 했다고 한다안그러면 결혼을 못한다고 한다.
여자들 마통 뚫고 하루 몇백 호캉스 & 오마카세 쳐먹고 빚을  ' 안정적 ' 이게 갚아줄 남자 찾는데.. 요즘 남자가 바보도 아니고 그런 여자 안만남.남자도 요즘 혼자 취미생활 하면서 즐길거리 있어서 결혼까지 생각 안하는거 같고 그냥 딱 연애정도만으로도 만족

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
5
"다 인스타 때문이라니까!" 전 세계 박제된 대한민국 / SBS / 모아보는 뉴스
없어졌으면 하는 문화. 1. 지나치게 돈이 많이 드는 결혼식.2. 결혼식에 강제 축의금.결혼은 부부가 하나 됐음을 알리고 축하 받는 자리일 뿐인데, 결혼식은 지나치게 비싸고 그 금액을 손님들에게 청구한다. 도대체 누굴 위한 결혼인가?
초중딩 시절 우리집 참 못살았는데 난 그걸 전혀 모르고 천진난만하게 자신감 넘치게 살았다. 아마 그때 SNS 있었으면 10대때 부터 우울증 걸렸을듯. 진짜 정승재 쌤 말 백배 맞음
90년대 초반생으로서 진짜 다행인것중 하나가 스마트폰, SNS, 유튜브 없던 시대에서 10대 어린시절 보냈다는거임. 남이랑 비교하고 허영심 부리는게 너무 컬쳐화 되버린 세상이다 ㄹㅇ
외국이 우리나라 까는 보도를 하는 건데도 속이 시원해진다 참 씁쓸하다
자기 형편에 맞게 사는게 이게 진짜 중요함
남들 다 하는거 안하면 이상한 사람으로 보는 문화도 없어졌으면 좋겠다.. 특히 결혼 관련해서 생략하고 싶은것도 많고 소규모로 하고싶은데 남들 시선을 아예 무시하고 살순없어서 다 맞춰서 해야된다는게...
우리나라는 이런 비난 좀 받아야 됩니다. 허세만 가득한 것들.
인스타 하면 너무 허세를 부추기고 나만 초라해지는거 같아 그만 뒀는데 너무 너무 너무 좋습니다. 인스타 오래하면 현실에 주력하며 사는게 아니라 허상에 나를 끼워 넣어 사는 느낌입니다. 정승제 강사님 말씀에 너무

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D66E73+48323]
	(No symbol) [0x00CF9661]
	(No symbol) [0x00C051D0]
	(No symbol) [0x00C319E6]
	(No symbol) [0x00C2A0FC]
	(No symbol) [0x00C4A8DC]
	(No symbol) [0x00C29BD6]
	(No symbol) [0x00C4AC34]
	(No symbol) [0x00C5CAC2]
	(No symbol) [0x00C4A6D6]
	(No symbol) [0x00C2847C]
	(No symbol) [0x00C2957D]
	GetHandleVerifier [0x00FCFD5D+2575277]
	GetHandleVerifier [0x0100F86E+2836158]
	GetHandleVerifier [0x010096DC+2811180]
	GetHandleVerifier [0x00DF41B0+626688]
	(No symbol) [0x00D0314C]
	(No symbol) [0x00CFF4B8]
	(No symbol) [0x00CFF59B]
	(No symbol) [0x00CF21B7]
	BaseThreadInitThunk [0x76C07D59+25]
	RtlInitializeExceptionChain [0x7709B79B+107]
	RtlClearBits [0x7709B71F+191]
